In [11]:
# ✅ Deutsch–Jozsa Algorithm (works on Qiskit 2.x, Python 3.12+)
# Includes oracle f(x)=x XOR 1, Bloch visualization, noise, and classical comparison

# --- Install (run once if not installed) ---
# !pip install qiskit qiskit-aer qiskit[visualization]

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
import matplotlib.pyplot as plt


# ---------------------------
# Oracle for f(x) = x XOR 1
# ---------------------------
def oracle_fx_xor_1(num_input_qubits=1):
    qc = QuantumCircuit(num_input_qubits + 1, name='oracle_f(x)=x XOR 1')
    anc = num_input_qubits
    for i in range(num_input_qubits):
        qc.cx(i, anc)
    qc.x(anc)
    return qc


# ---------------------------
# 1-qubit Deutsch–Jozsa + Bloch visualization
# ---------------------------
def deutsch_jozsa_1_qubit_with_bloch(oracle_circ):
    qc = QuantumCircuit(2, 1)
    qc.x(1)
    qc.h([0, 1])

    # Visualize after Hadamard
    sv_before = Statevector(qc)
    plot_bloch_multivector(sv_before)

    # Apply oracle
    qc.append(oracle_circ.to_instruction(), [0, 1])

    # Visualize after oracle
    sv_after_oracle = Statevector(qc)
    plot_bloch_multivector(sv_after_oracle)

    qc.h(0)
    qc.measure(0, 0)

    simulator = Aer.get_backend('aer_simulator')
    t_qc = transpile(qc, simulator)
    result = simulator.run(t_qc, shots=1024).result()
    counts = result.get_counts()
    plot_histogram(counts)
    return counts


# ---------------------------
# 2-qubit Oracle Builder
# ---------------------------
def oracle_from_truth_table(truth_table):
    n = len(next(iter(truth_table.keys())))
    qc = QuantumCircuit(n + 1, name='oracle_from_table')
    anc = n
    for s, val in truth_table.items():
        if val == 1:
            for i, bit in enumerate(s):
                if bit == '0':
                    qc.x(i)
            qc.mcx(list(range(n)), anc)
            for i, bit in enumerate(s):
                if bit == '0':
                    qc.x(i)
    return qc


# ---------------------------
# Truth Table for f(x0, x1) = x0 XOR x1 XOR 1
# ---------------------------
def truth_table_xor_plus_one(n):
    from itertools import product
    table = {}
    for bits in product('01', repeat=n):
        s = ''.join(bits)
        val = 0
        for ch in s:
            val ^= int(ch)
        val ^= 1
        table[s] = val
    return table


# ---------------------------
# Simple Noise Model
# ---------------------------
def make_simple_noise_model():
    noise_model = NoiseModel()
    error1 = depolarizing_error(0.001, 1)
    error2 = depolarizing_error(0.01, 2)
    noise_model.add_all_qubit_quantum_error(error1, ['h', 'x'])
    noise_model.add_all_qubit_quantum_error(error2, ['cx'])
    ro_err = ReadoutError([[0.99, 0.01], [0.02, 0.98]])
    noise_model.add_all_qubit_readout_error(ro_err)
    return noise_model


def run_with_noise(qc, noise_model, shots=1024):
    sim = Aer.get_backend('aer_simulator')
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc, shots=shots, noise_model=noise_model).result()
    return result.get_counts()


# ---------------------------
# Classical evaluator for constant vs balanced
# ---------------------------
def classical_evaluator(truth_func, n):
    from random import sample
    domain = list(range(2 ** n))
    sampled = sample(domain, len(domain))
    seen = set()
    queries = 0
    for x in sampled:
        queries += 1
        seen.add(truth_func(x))
        if len(seen) == 2:
            return {'decision': 'balanced', 'queries': queries}
    return {'decision': 'constant', 'queries': queries}


# ---------------------------
# Example Execution
# ---------------------------
if __name__ == '__main__':
    oracle1 = oracle_fx_xor_1(1)
    print("Running Deutsch–Jozsa for f(x)=x XOR 1...")
    counts = deutsch_jozsa_1_qubit_with_bloch(oracle1)
    print("Measurement counts:", counts)

    # 2-qubit oracle
    tt = truth_table_xor_plus_one(2)
    print("\\nTruth table for f(x0,x1)=x0 XOR x1 XOR 1:")
    for k in sorted(tt.keys()):
        print(k, '->', tt[k])

    # Classical vs Quantum comparison
    def f_int(x):
        return x ^ 1

    result = classical_evaluator(f_int, 1)
    print("\\nClassical evaluation:", result)
    print("Quantum Deutsch–Jozsa needs only 1 oracle call.")


Running Deutsch–Jozsa for f(x)=x XOR 1...
Measurement counts: {'1': 1024}
\nTruth table for f(x0,x1)=x0 XOR x1 XOR 1:
00 -> 1
01 -> 0
10 -> 0
11 -> 1
\nClassical evaluation: {'decision': 'balanced', 'queries': 2}
Quantum Deutsch–Jozsa needs only 1 oracle call.
